In [ ]:
import os
import random as rnd
import cv2
import numpy as np
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt

In [ ]:
# TODO Move into config.yaml
LOGO_URLS = ["https://sberdevices.ru/file/5c38bf50-e3f9-4c4e-8649-b773c09d6660/sber-logo.png",]
BG_URLS = ["https://w.forfun.com/fetch/70/700b81460b8ff6b65af31fd80bda2f7b.jpeg",]

In [ ]:
SAMPLE_INDEX = 0

In [ ]:
class ImageHolder:
    def __init__(self):
        self.logo_index = 0
        self.bg_index = 0
        self.logos = list()
        self.bgs = list()

    def save_image_from_url(self, url: str, im_type: bool) -> None:
        arr = np.asarray(bytearray(urlopen(url).read()), dtype=np.uint8)
        img = cv2.imdecode(arr, -1)
        name = f"bg/sample_{self.bg_index}.png" if im_type else f"lg/sample_{self.logo_index}.png"
        cv2.imwrite(name, img)
        if im_type:
            self.bgs.append(cv2.imread(name))
        else:
            self.logos.append(cv2.imread(name))

    def download_and_save_from_config(self):
        os.makedirs("bg", exist_ok=True)
        os.makedirs("lg", exist_ok=True)
        for i in LOGO_URLS:
            self.save_image_from_url(i, False)
            self.logo_index += 1
        for i in BG_URLS:
            self.save_image_from_url(i, True)
            self.bg_index += 1

In [ ]:
holder = ImageHolder()
holder.download_and_save_from_config()

In [ ]:
class ImageGenerator:
    def __init__(self, logo: np.array, samples_count: int):
        self.logo = logo
        self.samples_count = samples_count
        self.resized_logos = list()

    def main(self):
        global SAMPLE_INDEX
        os.makedirs("sm", exist_ok=True)
        for i in range(self.samples_count):
            scale = rnd.randint(3, 10)
            dim = (int(self.logo.shape[1]/scale),int(self.logo.shape[0]/scale))
            self.resized_logos.append(cv2.resize(self.logo, dim, interpolation = cv2.INTER_AREA))
        for i in self.resized_logos:
            bg = rnd.choice(holder.bgs)
            roi = self.make_logo_roi(i, np.copy(bg))
            self.make_sample(np.copy(bg), roi, f"sample_{SAMPLE_INDEX}")
            SAMPLE_INDEX += 1

    @staticmethod
    def make_logo_roi(logo: np.array, bg: np.array) -> tuple:
        x_offset, y_offset = rnd.randint(0, bg.shape[1] - logo.shape[1]), rnd.randint(0, bg.shape[0] - logo.shape[0])
        x_end, y_end = (x_offset + logo.shape[1], y_offset + logo.shape[0])
        roi = bg[y_offset:y_offset+logo.shape[0], x_offset:x_offset+logo.shape[1]]

        logo_gray = cv2.cvtColor(logo, cv2.COLOR_RGB2GRAY)
        ret, mask = cv2.threshold(logo_gray, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        temp_bg = cv2.bitwise_or(roi,roi,mask = mask)
        mask_inv = cv2.bitwise_not(logo_gray)
        fg = cv2.bitwise_and(logo,logo, mask=mask_inv)
        final_roi = cv2.add(temp_bg,fg)

        return y_offset, y_end, x_offset, x_end, final_roi

    @staticmethod
    def make_sample(bg: np.array, roi: tuple, save_path: str) -> None:
        bg[roi[0] : roi[1], roi[2] : roi[3]]= roi[4]
        label = open(f"sm/{save_path}.txt", "w+")
        x_center = ((roi[2] + roi[3]) / 2) / bg.shape[1]
        y_center = ((roi[0] + roi[1]) / 2) / bg.shape[0]
        norm_width = (roi[3] - roi[2]) / bg.shape[1]
        norm_height = (roi[1] - roi[0]) / bg.shape[0]
        label.write(f"0 {x_center} {y_center} {norm_width} {norm_height}\n")
        label.close()

        cv2.imwrite(f"sm/{save_path}.png", bg)



In [ ]:
generator = ImageGenerator(holder.logos[0], 4)
generator.main()